In [1]:
using Pkg; for p in ("Gym","Plots"); haskey(Pkg.installed(),p) || Pkg.add(p); end
Pkg.add("PyCall")
ENV["GYM_ENVS"]="atari:classic_control:box2d"
ENV["PYTHON"]="/usr/local/bin/python3"
Pkg.build("PyCall"); Pkg.build("Gym")
using Gym, PyCall, Knet, Base.Iterators
np = pyimport("numpy")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
  Building Conda ─→ `~/.julia/packages/Conda/kLXeC/deps/build.log`
  Building PyCall → `~/.julia/packages/PyCall/ttONZ/deps/build.log`
  Building Conda ─→ `~/.julia/packages/Conda/kLXeC/deps/build.log`
  Building PyCall → `~/.julia/packages/PyCall/ttONZ/deps/build.log`
  Building Gym ───→ `~/.julia/packages/Gym/QZ2GY/deps/build.log`


PyObject <module 'numpy' from '/usr/local/lib/python3.7/site-packages/numpy/__init__.py'>

In [2]:
Pkg.add("Images"); using Images; Pkg.add("Colors"); using Colors
Pkg.add("ImageTransformations"); using ImageTransformations

 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


In [3]:
mutable struct Conv 
    w 
    b 
    f 
    p 
end
(c::Conv)(x) = c.f.(pool(conv4(c.w, dropout(x,c.p)) .+ c.b))
Conv(w1::Int,w2::Int,cx::Int,cy::Int, f=relu;pdrop=0) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop)

Conv

In [4]:
mutable struct Linear; w; b; end

function Linear(inputsize::Int, outputsize::Int)
    w = param(outputsize, inputsize)
    b = param0(outputsize)
    Linear(w,b)
end
function (l::Linear)(x)
    l.w * mat(x) .+ l.b
end

In [5]:
mutable struct Projection; w; end

function Projection(inputsize::Int, outputsize::Int)
    w = param(outputsize, inputsize)
    Projection(w)
end
function (p::Projection)(x)
    p.w * x
end

In [6]:
function mse_loss(arr1, arr2)
    sum((arr1-vec(arr2)).^2)/length(arr1)
end

mse_loss (generic function with 1 method)

In [7]:
mutable struct DQN
    conv1
    conv2
    conv3
    fc1
    output
end

function DQN(filter_size1, filter_amount_1, filter_size2, filter_amount_2, filter_size3, filter_amount_3, input_channels::Int, 
    hidden_size1::Int, hidden_size2::Int, action_space_size::Int)
    conv1 = Conv(filter_size1, filter_size1, input_channels, filter_amount_1)
    conv2 = Conv(filter_size2, filter_size2, filter_amount_1, filter_amount_2)
    conv3 = Conv(filter_size3, filter_size3, filter_amount_2, filter_amount_3)
    fc1 = Linear(hidden_size1, hidden_size2)
    output = Projection(hidden_size2, action_space_size)     
    DQN(conv1, conv2, conv3, fc1, output)
end

function (model::DQN)(x)
    model.output(relu.(model.fc1(relu.(model.conv3(relu.(model.conv2(relu.(model.conv1(x)))))))))
end


function (model::DQN)(x, indexes, q_star)
    mse_loss(model(x)[indexes],q_star)
end

In [8]:
mutable struct ReplayMemory
    capacity
    memory
    push_count
    batchsize
end


function ReplayMemory(capacity::Int, batchsize::Int)
    push_count = 0
    memory = []
    ReplayMemory(capacity, memory, push_count, batchsize)
end


function PushToReplayMemory!(r::ReplayMemory, e)
    
    if(length(r.memory) < r.capacity)
        push!(r.memory, e)
    else
        r.memory[(r.push_count%r.capacity)+1] = e
    end
    r.push_count += 1
end

function CanProvideSample(r::ReplayMemory)
    return r.push_count >= r.batchsize
end

function Sample(r::ReplayMemory)
    return r.memory[rand(1:min(r.push_count, r.capacity),r.batchsize)]
end

Sample (generic function with 1 method)

In [9]:
struct EpsilonGreedyStrategy
    max
    min
    decay
end

function GetExplorationRate(e::EpsilonGreedyStrategy, current_step::Int)
    e.min + ((e.max-e.min) * exp(-1*current_step*e.decay))
end

GetExplorationRate (generic function with 1 method)

In [10]:
mutable struct Agent
    current_step
    strategy
    num_actions
end

function Agent(strategy::EpsilonGreedyStrategy, num_actions::Int)
    current_step = 0
    return Agent(current_step, strategy, num_actions)
end

function GetAction(a::Agent, state, policy_net, env; training = true)
    a.current_step += 1
    ϵ = GetExplorationRate(a.strategy, a.current_step)
    
    if(training && ϵ > rand())
    #if(false)
        return rand(0:env.action_space.n-1)
    else
        x,y,z = size(state)
        return argmax(policy_net(reshape(state,(x,y,z,1))))[1]-1
    end 
end


function setCurrentStep!(agent, step)
    agent.current_step = step
end

setCurrentStep! (generic function with 1 method)

In [11]:
reward = 0
num_episodes = 10
batch_size = 32
γ = 0.95
ϵ_start = 0.9
ϵ_decay = 0.005 
ϵ_end = 0.01
target_update = 10
memory_size = 10000
lr = 0.00025
image_height = 84 
image_width = 84
input_channels = 4
hidden_size1 = 3136
hidden_size2 = 512
moving_average_length = 1
gradientclip = 1

1

In [12]:
filter_size1 = 8
filter_amount_1 = 32
filter_size2 = 4
filter_amount_2 = 64
filter_size3 = 3
filter_amount_3 = 64 

64

In [13]:
cropfromtop = 0.0
cropfrombottom = 0.0
target_height = 84
target_width = 84

84

In [14]:
function GetPreProcessedImageArray(img, cropfromtop::Float64, cropfrombottom::Float64, target_height::Int, target_width::Int)
    img = img/255
    height, width, channels = size(img)
    state_image = permutedims(img,(3,1,2))
    state_image2 = (state_image[:,Int(height*cropfromtop)+1:Int((1-cropfrombottom)*height),:])
    grayscale = Gray.(state_image2)
    reshape(Float32.(imresize(grayscale, (1,target_height, target_width))), (target_width,target_height))
end
    

GetPreProcessedImageArray (generic function with 1 method)

In [15]:
function extract_tensors(experiences, target_width, target_height, input_channels, batchsize)
    tensors = reshape(collect(flatten(experiences)), (4,batchsize))
    states = reshape(cat1d(tensors[1,:]...), (target_width, target_height, input_channels, batchsize))
    actions = tensors[2,:]
    next_states = reshape(cat1d(tensors[3,:]...), (target_width, target_height, input_channels, batchsize))
    rewards = tensors[4,:]
    return states, actions, next_states, rewards
end

extract_tensors (generic function with 1 method)

In [16]:
function GetMovingAverage(collection, n)
    len = length(collection)
    if(n>len)
        return 0
    else
        return sum(collection[len-n+1:end])/n
    end
end

GetMovingAverage (generic function with 1 method)

In [17]:
env = GymEnv("Pong-v0")

GymEnv("Pong-v0", Gym.Spec("Pong-v0", nothing, false, Dict{Any,Any}("wrapper_config.TimeLimit.max_episode_steps" => 10000), 10000), DiscreteS(6), BoxS(UInt8[0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00; … ; 0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00]

UInt8[0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00; … ; 0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00]

UInt8[0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00; … ; 0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00], UInt8[0xff 0xff … 0xff 0xff; 0xff 0xff … 0xff 0xff; … ; 0xff 0xff … 0xff 0xff; 0xff 0xff … 0xff 0xff]

UInt8[0xff 0xff … 0xff 0xff; 0xff 0xff … 0xff 0xff; … ; 0xff 0xff … 0xff 0xff; 0xff 0xff … 0xff 0xff]

UInt8[0xff 0xff … 0xff 0xff; 0xff 0xff … 0xff 0xff; … ; 0xff 0xff … 0xff 0xff; 0xff 0xff … 0xff 0xff], (210, 160, 3)), (-Inf, Inf), PyObject <TimeLimit<AtariEnv<Pong-v0>>>)

In [18]:
episode_rewards = []
replay_memory = ReplayMemory(memory_size, batch_size)
policy_network = DQN(filter_size1, filter_amount_1, filter_size2, filter_amount_2, filter_size3, filter_amount_3, input_channels, hidden_size1, hidden_size2, env.action_space.n)
policy_network = Knet.load("policyv2.jld2","model")
target_network = deepcopy(policy_network)
strategy = EpsilonGreedyStrategy(ϵ_start, ϵ_end, ϵ_decay)
agent = Agent(strategy, env.action_space.n)
#setCurrentStep!(agent, 27000)
total_reward_per_episode = 0

0

In [19]:
#for training

for i = 1:num_episodes
    done = false
    reset!(env) 
    current_state = zeros(Float32, 84,84,4)
    if (gpu()>=0)
        current_state = KnetArray(current_state)
    end
    println("episode: ", i, " 100MA: ", GetMovingAverage(episode_rewards,moving_average_length))
    
    total_reward_per_episode = 0
    time_step = 0
    while(!done) 
        time_step += 1
        #println("timestep: ", time_step)
        action = GetAction(agent, current_state, policy_network, env)
        observation, reward, done, information = step!(env, action) 
        if(done)
            next_state = zeros(Float32, 84,84,4)
            if (gpu()>=0)
                next_state = KnetArray(next_state)
            end
        else
            img = GetPreProcessedImageArray(observation, cropfromtop, cropfrombottom, target_height, target_width)
            img = reshape(img, (target_width, target_height, 1))
            if (gpu()>=0)
                img = KnetArray(img)
            end
            next_state = cat(current_state, img; dims=3)[:,:,2:5]
        end
        
        total_reward_per_episode += reward
        PushToReplayMemory!(replay_memory, [current_state, action, next_state, reward])
        current_state = next_state
        if(CanProvideSample(replay_memory))
            experiences = Sample(replay_memory)
            states, actions, next_states, rewards = extract_tensors(experiences, image_width, image_height, input_channels, batch_size)
            if (gpu()>=0)
                states = KnetArray(states)
                actions = KnetArray(Int.(actions))
                next_states = KnetArray(next_states)
                rewards = KnetArray(Float32.(rewards))
            end
            indexes = collect(0:batch_size-1)*env.action_space.n + Array(actions) .+ 1
            global target_network
            q_star = maximum(target_network(next_states), dims=1) * γ + rewards' 
            loss_before = mse_loss(policy_network(states)[indexes], q_star)
            tup = (states, indexes, q_star)
            list = [tup]
            #adam!(policy_network, take(cycle(list),1), lr=lr, gclip=gradientclip)
            adam!(policy_network, take(cycle(list),1), lr=lr)
            loss_after = mse_loss(policy_network(states)[indexes], q_star)
           
            println("timestep: ", time_step, " loss b4: ", loss_before, " loss after: ", loss_after)
        end
        
        if(done)
            push!(episode_rewards, total_reward_per_episode)
        end
        if(agent.current_step%target_update == 0)
            target_network = deepcopy(policy_network)
        end
    end
    
end

Knet.save("policyv2.jld2","model",policy_network)

episode: 1 100MA: 0

Stacktrace:
 [1] Type at ./boot.jl:339 [inlined]
 [2] AutoGrad.Result(::Array{Float32,4}, ::Function, ::Tuple{typeof(+),AutoGrad.Result{Array{Float32,4}},Param{Array{Float32,4}}}, ::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at /Users/deniz/.julia/packages/AutoGrad/pTNVv/src/core.jl:111
 [3] #forw#1(::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}, ::typeof(AutoGrad.forw), ::Function, ::Function, ::Vararg{Any,N} where N) at /Users/deniz/.julia/packages/AutoGrad/pTNVv/src/core.jl:72
 [4] forw at /Users/deniz/.julia/packages/AutoGrad/pTNVv/src/core.jl:65 [inlined]
 [5] broadcasted(::typeof(+), ::AutoGrad.Result{Array{Float32,4}}, ::Param{Array{Float32,4}}) at ./none:0
 [6] (::Conv)(::AutoGrad.Result{Array{Float32,4}}) at ./In[3]:7
 [7] (::DQN)(::Array{Float32,4}) at ./In[7]:20
 [8] (::DQN)(::Array{Float32,4}, ::Array{Int64,1}, ::Array{Float64,2}) at ./In[7]:25
 [9] (::getfield(Knet, Symbol("##695#696")){Knet.Minimize{B

InterruptException: InterruptException:

In [19]:
#for demo


policy_network = Knet.load("policyv2.jld2","model")


for i = 1:2
    done = false
    reset!(env) 
    current_state = zeros(Float32, 84,84,4)    
    total_reward_per_episode = 0
    render(env)
    
    while(!done) 
        
        action = GetAction(agent, current_state, policy_network, env; training = false)
        observation, reward, done, information = step!(env, action) 
        render(env)
        
        if(done)
            next_state = zeros(Float32, 84,84,4)
        
        else
            img = GetPreProcessedImageArray(observation, cropfromtop, cropfrombottom, target_height, target_width)
            img = reshape(img, (target_width, target_height, 1))
            next_state = cat(current_state, img; dims=3)[:,:,2:5]
        end
        
        total_reward_per_episode += reward
        current_state = next_state
       
            
    
        if(done)
            println("reward is: ", total_reward_per_episode)
        end
    end
    
end

reward is: -16.0
reward is: -18.0
